<a href="https://colab.research.google.com/github/bf777/MesoNet/blob/master/mesonet_demo_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MesoNet
Welcome to MesoNet, a toolbox for segmenting mesoscale calcium images! This notebook will take you through all the steps needed to process your own calcium imaging dataset using our models.

In [ ]:
# Clone MesoNet repository
!git clone https://github.com/bf777/MesoNet.git

## Prepare inputs

In [ ]:
!mkdir /content/mesonet_inputs/

Now, we will copy sample data (11 images) from the git repository to the inputs folder. If, instead, you would like to upload your own images create a folder inside `mesonet_inputs` and put your images inside that folder; then, skip the next cell. 

In [ ]:
# If you have your own input data that you've put into a subfolder under
# mesonet_inputs, don't run this cell.
%cp -r MesoNet/tests/test_input/ mesonet_inputs/

In [ ]:
%cd MesoNet/

/content/MesoNet


In [ ]:
# Utility to install mesonet package and associated requirements (will be replaced with pip later)
!pip install matplotlib==3.1.3

# Install DeepLabCut for pose estimation
!pip install deeplabcut

# Install MesoNet
!python setup.py install

# Reinstall OpenCV to address compatibility issue
!pip install opencv-python==4.4.0.46

# Reinstall h5py
!pip install h5py==2.10.0

# pip install mesonet

Because of how DeepLabCut operates, you now need to restart your runtime (under the Runtime menu).

In [ ]:
# Use tensorflow 1.x (supported by DeepLabCut)
%tensorflow_version 1.x
import tensorflow
import os
print(tensorflow.__version__)

1.15.2


We will now pull information from the OSF repository containing the DeepLabCut and U-Net models for our code.

In [ ]:
!pip install osfclient

In [ ]:
%cd /content/
!osf -p svztu fetch 6_Landmark_estimation_model/atlas-DongshengXiao-2020-08-03.zip mesonet_inputs/atlas-DongshengXiao-2020-08-03.zip

In [ ]:
!unzip -q mesonet_inputs/atlas-DongshengXiao-2020-08-03.zip -d /content/mesonet_inputs

In [ ]:
%cd /content/
!osf -p svztu fetch 7_U-Net_model/DongshengXiao_brain_bundary.hdf5 MesoNet/mesonet/models/DongshengXiao_brain_bundary.hdf5

Now, input the information about your input and output images, as well as the U-Net and DeepLabCut models that you would like to use. The default values will use the test data that we've included in the MesoNet git repository (in `MesoNet/mesonet/tests/test_input`). If you're using your own input data, replace `input_filename` below with the name of the folder in `mesonet_inputs` containing your input data.

In [ ]:
#@title Input information for the model
input_filename = 'test_input'  #@param {type: "string"}
output_filename = 'mesonet_outputs'  #@param {type: "string"}
model_name = 'DongshengXiao_brain_bundary.hdf5' #@param {type: "string"}
dlc_model_name = 'atlas-DongshengXiao-2020-08-03'  #@param {type: "string"}

In [ ]:
# Set up filepaths based on your inputs
input_file = os.path.join('/content','mesonet_inputs', input_filename)
output_file = os.path.join('/content', output_filename)
model = os.path.join('/content', 'MesoNet', 'mesonet', 'models', model_name)
dlc_config = os.path.join('/content','mesonet_inputs', dlc_model_name, 
                          'config.yaml')

In [ ]:
!mkdir '/content/mesonet_outputs/'

Now that we've told Colab where to find the input and output folders, let's define the configuration file!

## Configure MesoNet

In [ ]:
# Set this environment variable to help MesoNet find the git repo location
os.environ["MESONET_GIT"]='/content/MesoNet/mesonet/'

In [ ]:
# We need to make sure that DeepLabCut doesn't run with a GUI (which isn't
# supported in Colab).
os.environ["DLClight"]="True"

# Import mesonet and define the configuration file according to your chosen
# inputs
import mesonet
config_file = mesonet.config_project(input_file, output_file, 'test',
                                     use_voxelmorph=False, 
                                     mat_save=False, config=dlc_config, 
                                     model=model)

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


Using TensorFlow backend.


/content/MesoNet/mesonet/


The config file (by default in `mesonet_outputs`) contains information about how MesoNet will run. We'll be using this config file as an input to the last two functions needed to run MesoNet.

In [ ]:
config_file

'/content/mesonet_outputs/mesonet_test_config.yaml'

Next, we'll use the U-net model to identify the outer edges of the cortex:

## Run MesoNet

### Predict regions

In [ ]:
%cd /content/
mesonet.predict_regions(config_file)

/content
/content/MesoNet/mesonet/models/DongshengXiao_brain_bundary.hdf5
/content/mesonet_inputs/test_input
12/12 [==============================] - 11s 942ms/step


Lastly, we will identify and use cortical landmarks to align the atlas to the brain:

### Predict landmarks

In [ ]:
mesonet.predict_dlc(config_file)

12
/content/mesonet_inputs/test_input/0.png
/content/mesonet_inputs/test_input/1.png
/content/mesonet_inputs/test_input/2.png
/content/mesonet_inputs/test_input/3.png
/content/mesonet_inputs/test_input/4.png
/content/mesonet_inputs/test_input/5.png
/content/mesonet_inputs/test_input/6.png
/content/mesonet_inputs/test_input/7.png
/content/mesonet_inputs/test_input/8.png
/content/mesonet_inputs/test_input/9.png
/content/mesonet_inputs/test_input/10.png
/content/mesonet_inputs/test_input/11.png
Using snapshot-1030000 for model /content/mesonet_inputs/atlas-DongshengXiao-2020-08-03/dlc-models/iteration-2/atlasAug3-trainset95shuffle1
Initializing ResNet


  0%|          | 0/12 [00:00<?, ?it/s]

Analyzing all the videos in the directory...
Starting to analyze %  /content/mesonet_outputs/dlc_output/tmp_video.mp4
/content/mesonet_outputs/dlc_output  already exists!
Loading  /content/mesonet_outputs/dlc_output/tmp_video.mp4
Duration of video [s]:  0.4 , recorded with  30.0 fps!
Overall # of frames:  12  found with (before cropping) frame dimensions:  512 512
Starting to extract posture


20it [00:01, 12.62it/s]


Saving results in /content/mesonet_outputs/dlc_output...
Saving csv poses!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
/content/mesonet_outputs/dlc_output  already exists!
Starting to process video: /content/mesonet_outputs/dlc_output/tmp_video.mp4
Loading /content/mesonet_outputs/dlc_output/tmp_video.mp4 and data.
No filtered data file found in /content/mesonet_outputs/dlc_output for video tmp_video and scorer DLC_resnet50_atlasAug3shuffle1_1030000.
Landmark prediction complete!
Performing first transformation of atlas 0...
Performing second transformation of atlas 0...
Performing first transformation of atlas 1...
Performing second transformation of atlas 1...
Performing first transformation of atlas 2...
Performing second t

Congratulations, you're all done! You can now check the outputs in the `mesonet_outputs` folder. The segmented brain data can be found in `mesonet_outputs/output_mask`!